In [2]:
!wget https://raw.githubusercontent.com/Munazaa/Flood-Mapping-Case/main/requirements.txt



--2024-12-09 13:52:22--  https://raw.githubusercontent.com/Munazaa/Flood-Mapping-Case/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]      40  --.-KB/s    in 0s      

2024-12-09 13:52:22 (1.82 MB/s) - ‘requirements.txt’ saved [40/40]



In [3]:
!pip install -r requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.0 MB/s eta 0:00:00


In [ ]:
import ee

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

In [ ]:
#Define AOI (Area of Interest)
aoi = ee.Geometry.Polygon(
    [[[35.53377589953368, -19.6674648789114],
      [34.50106105578368, -18.952058786515526],
      [33.63314113390868, -19.87423907259203],
      [34.74825343859618, -20.61123742951084]]]
)

# Define parameters
before_start = '2018-10-05'
before_end = '2018-10-26'
after_start = '2018-10-27'
after_end = '2018-11-20'
polarization = "VV"
pass_direction = "ASCENDING"
difference_threshold = 1.25

# Load Sentinel-1 collection
collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
    .filter(ee.Filter.eq('orbitProperties_pass', pass_direction)) \
    .filter(ee.Filter.eq('resolution_meters', 10)) \
    .filterBounds(aoi) \
    .select(polarization)

before_collection = collection.filterDate(before_start, before_end)
after_collection = collection.filterDate(after_start, after_end)

# Create mosaics and apply focal mean (smoothing)
before = before_collection.mosaic().clip(aoi).focal_mean(50, 'circle', 'meters')
after = after_collection.mosaic().clip(aoi).focal_mean(50, 'circle', 'meters')

# Calculate the difference and apply threshold
difference = after.divide(before)
flood_mask = difference.gt(difference_threshold)

# Refine flood mask using JRC Global Surface Water
swater = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('seasonality')
swater_mask = swater.gte(10)
flooded = flood_mask.where(swater_mask, 0).updateMask(flood_mask)

# Calculate connected components to reduce noise
connections = flooded.connectedPixelCount(8)
flooded = flooded.updateMask(connections.gte(8))

# Add slope mask using DEM
DEM = ee.Image('WWF/HydroSHEDS/03VFDEM')
slope = ee.Terrain.slope(DEM)
flooded = flooded.updateMask(slope.lt(17))

# Add flooded areas to map
print("Flooded areas calculated.")

# Define Sentinel-2 functions
def calculate_ndwi(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return ndwi.updateMask(ndwi).clip(aoi)

# Load Sentinel-2 collection
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterBounds(aoi) \
    .filterDate(before_start, after_end) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

s2_before_collection = sentinel2.filterDate(before_start, before_end).map(calculate_ndwi)
s2_after_collection = sentinel2.filterDate(after_start, after_end).map(calculate_ndwi)

if s2_before_collection.size().getInfo() > 0 and s2_after_collection.size().getInfo() > 0:
    s2_before = s2_before_collection.median()
    s2_after = s2_after_collection.median()
    s2_flood = s2_after.subtract(s2_before).gt(0.1).rename('Flood')

    print("Sentinel-2 Flood Layer calculated.")

    # Export Sentinel-2 raster
    task_s2_raster = ee.batch.Export.image.toDrive(
        image=s2_flood,
        description='Sentinel-2_Flood_Raster',
        fileNamePrefix='Sentinel2_Flood',
        region=aoi,
        scale=10,
        maxPixels=1e10
    )
    task_s2_raster.start()

    # Convert flood raster to vectors
    s2_flood_vec = s2_flood.reduceToVectors(
        scale=10,
        geometryType='polygon',
        geometry=aoi,
        eightConnected=False,
        bestEffort=True
    )

    # Export Sentinel-2 vector
    task_s2_vector = ee.batch.Export.table.toDrive(
        collection=s2_flood_vec,
        description='Sentinel-2_Flood_Vector',
        fileFormat='SHP'
    )
    task_s2_vector.start()
else:
    print("No valid Sentinel-2 data available for the specified periods.")

# Export Sentinel-1 Flood Raster
task_s1_raster = ee.batch.Export.image.toDrive(
    image=flooded,
    description='Sentinel-1_Flood_Raster',
    fileNamePrefix='Sentinel1_Flood',
    region=aoi,
    scale=10,
    maxPixels=1e10
)
task_s1_raster.start()

# Convert Sentinel-1 flood raster to vectors
flooded_vec = flooded.reduceToVectors(
    scale=10,
    geometryType='polygon',
    geometry=aoi,
    eightConnected=False,
    bestEffort=True
)

# Export Sentinel-1 Flood Vector
task_s1_vector = ee.batch.Export.table.toDrive(
    collection=flooded_vec,
    description='Sentinel-1_Flood_Vector',
    fileFormat='SHP'
)
task_s1_vector.start()

print("Export tasks started.")
